DATASET USED:  https://data.mendeley.com/datasets/rscbjbr9sj/3

In [ ]:
# Importing libraries
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.data import AUTOTUNE

train_dir = 'C:/chest_xray/train'


#check for GPU availability (if available, it uses CUDA)

if tf.test.gpu_device_name():
    print('GPU is available.')
else:
    print('No GPU detected.')

No GPU detected.


In [ ]:
#define the training data

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    subset="training",
    validation_split=0.2, #percentage of data used for training
    seed=123,
    image_size=(180, 180),
    batch_size=32,#how many images used per epoch before parameters are updated
    color_mode="grayscale")


Found 5232 files belonging to 2 classes.
Using 4186 files for training.


In [ ]:
test_dir =

In [ ]:
# Testing or Validation split
val_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    subset="validation",
    validation_split=0.2,
    seed=123,
    image_size=(180,180),
    batch_size=32,
    color_mode="grayscale")


Found 5232 files belonging to 2 classes.
Using 1046 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['NORMAL', 'PNEUMONIA']


In [ ]:
from tensorflow.keras import regularizers

num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(180, 180, 1)),
  layers.Conv2D(16, 3, strides=2, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu', activity_regularizer=regularizers.L2(0.01)),
  layers.BatchNormalization(),
  layers.Dense(1, activation='sigmoid', activity_regularizer=regularizers.L2(0.01))
])

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 180, 180, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 90, 90, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 45, 45, 16)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 32400)             0         
                                                                 
 dense (Dense)               (None, 128)               4147328   
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                         

In [ ]:
epochs=7
model.fit(
train_ds,
validation_data=val_ds,
epochs=epochs
)


Epoch 1/7
131/131 [==============================] - 43s 317ms/step - loss: 0.1772 - accuracy: 0.9577 - val_loss: 0.2928 - val_accuracy: 0.9187
Epoch 2/7
131/131 [==============================] - 45s 333ms/step - loss: 0.1663 - accuracy: 0.9611 - val_loss: 0.4516 - val_accuracy: 0.8480
Epoch 3/7
131/131 [==============================] - 49s 362ms/step - loss: 0.1534 - accuracy: 0.9658 - val_loss: 0.1777 - val_accuracy: 0.9618
Epoch 4/7
131/131 [==============================] - 53s 390ms/step - loss: 0.1406 - accuracy: 0.9730 - val_loss: 0.2181 - val_accuracy: 0.9426
Epoch 5/7
107/131 [=======================>......] - ETA: 8s - loss: 0.1321 - accuracy: 0.9766


KeyboardInterrupt



In [ ]:
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
from PIL import Image
import io


upload_button = widgets.FileUpload(accept='.jpg,.png', multiple=False)
image_output = widgets.Output()
prediction_output = widgets.Output()

# Define prediction function
def predict_image(change):
    image_output.clear_output()
    prediction_output.clear_output()

    if len(upload_button.value) == 0:
        with prediction_output:
            print("Please upload an image.")
        return

    uploaded_image = upload_button.value[0]
    image_content = uploaded_image['content']
    image = Image.open(io.BytesIO(image_content)).convert('L')
    image = image.resize((180, 180))
    img_array = tf.keras.utils.img_to_array(image)
    img_array = tf.expand_dims(img_array, 0)

    with image_output:
        display(image)

    with prediction_output:
        predictions = model.predict(img_array)
        score = float(predictions[0])

    print(f"This image is {100 * (1-score):.2f}% normal and {100 * score:.2f}% pneumonia.")

# Attach function to button click event
upload_button.observe(predict_image, names='value')

# Display widgets
display(widgets.Label("Upload an image:"))
display(upload_button)
display(image_output)
display(widgets.Label("Prediction:"))
display(prediction_output)

Label(value='Upload an image:')

FileUpload(value=(), accept='.jpg,.png', description='Upload')

Output()

Label(value='Prediction:')

Output()

This image is 15.31% normal and 84.69% pneumonia.
This image is 5.12% normal and 94.88% pneumonia.
